<a href="https://colab.research.google.com/github/Thabet-Chaaouri/Tokenizers-Datasets/blob/main/HF_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 5.0 MB/s 
     |████████████████████████████████| 163 kB 55.5 MB/s 
     |████████████████████████████████| 212 kB 69.7 MB/s 
     |████████████████████████████████| 115 kB 69.3 MB/s 
     |████████████████████████████████| 127 kB 70.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 52.6 MB/s 


In [ ]:
from datasets import load_dataset
import html
from transformers import AutoTokenizer, DataCollatorWithPadding
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam

In [ ]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

data_files={"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

--2022-10-03 07:01:43--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/x-httpd-php]
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip    100%[===================>]  41.00M  28.9MB/s    in 1.4s    

2022-10-03 07:01:45 (28.9 MB/s) - ‘drugsCom_raw.zip’ saved [42989872/42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [ ]:
drug_dataset=drug_dataset.rename_column("Unnamed: 0", "review_id")
drug_dataset=drug_dataset.filter(lambda x: x["condition"] is not None)
drug_dataset=drug_dataset.map(lambda batch: {"condition" : [x.lower() for x in batch["condition"]]}, batched=True)
drug_dataset = drug_dataset.map(lambda batch: {"review_len" : [len(x.split()) for x in batch["review"]] }, batched=True)
drug_dataset=drug_dataset.filter(lambda x: x["review_len"] > 30)
drug_dataset = drug_dataset.map(lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched=True)

  0%|          | 0/162 [00:00<?, ?ba/s]

  0%|          | 0/54 [00:00<?, ?ba/s]

In [ ]:
drug_dataset.set_format("pandas")
df = drug_dataset["train"][:]
drug_dataset.reset_format()

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)

%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
tokenized_dataset=tokenized_dataset.filter(lambda x: x["condition"] in list(df.condition.value_counts().index[:28]))

  0%|          | 0/139 [00:00<?, ?ba/s]

  0%|          | 0/47 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset_clean = tokenized_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
tokenized_dataset_clean["validation"] = tokenized_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
tokenized_dataset_clean["test"] = tokenized_dataset["test"]

In [ ]:
tokenized_dataset_clean=tokenized_dataset_clean.class_encode_column("condition")

Flattening the indices:   0%|          | 0/76 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/76 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/8 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/19 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/19 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/32 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/32 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_len', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 75640
    })
    validation: Dataset({
        features: ['review_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_len', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18910
    })
    test: Dataset({
        features: ['review_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_len', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 31515
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_dataset_clean["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["condition"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_dataset_clean["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["condition"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_dataset_clean["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["condition"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=28)
batch_size = 8
num_epochs = 3

num_train_steps = len(tf_train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = Adam(learning_rate=lr_scheduler)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

In [ ]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset,)

9455/9455 [==============================] - 3626s 381ms/step - loss: 0.7984 - accuracy: 0.7543 - val_loss: 0.5465 - val_accuracy: 0.8170


In [ ]:
import numpy as np
preds = model.predict(tf_validation_dataset)["logits"]
class_preds = np.argmax(preds, axis=1)
truth=np.array(tokenized_dataset_clean["test"]["condition"])

In [ ]:
accuracy=(truth==class_preds).sum()/len(truth)

In [ ]:
accuracy

0.8146279549420911